k-cross validation

In [2]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score



In [3]:
df=pd.read_csv(r'C:\Users\Abdulmuxtor\Downloads\placementdata_with_faculty.csv')

In [4]:
df.head()

,StudentID,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty
0,1,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced,Business Administration
1,2,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed,Computer Science
2,3,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced,Computer Science
3,4,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed,Civil Engineering
4,5,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed,Electrical Engineering


In [5]:
df.drop(columns=['StudentID'],inplace=True)

In [6]:
df.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty
0,7.5,1,1,1,65,4.4,No,No,61,79,NotPlaced,Business Administration
1,8.9,0,3,2,90,4.0,Yes,Yes,78,82,Placed,Computer Science
2,7.3,1,2,2,82,4.8,Yes,No,79,80,NotPlaced,Computer Science
3,7.5,1,1,2,85,4.4,Yes,Yes,81,80,Placed,Civil Engineering
4,8.3,1,2,2,86,4.5,Yes,Yes,74,88,Placed,Electrical Engineering


data prerocessing

missing values

In [7]:
df.isnull().sum()

CGPA                         0
Internships                  0
Projects                     0
Workshops/Certifications     0
AptitudeTestScore            0
SoftSkillsRating             0
ExtracurricularActivities    0
PlacementTraining            0
SSC_Marks                    0
HSC_Marks                    0
PlacementStatus              0
faculty                      0
dtype: int64

In [8]:
for col in df.columns:
    if df[col].dtype=='object':
        df[col].fillna(df[col].mode()[0],inplace=True)
    else:
        df[col].fillna(df[col].mode(),inplace=True)

C:\Users\Abdulmuxtor\AppData\Local\Temp\ipykernel_8\4165299538.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode(),inplace=True)
C:\Users\Abdulmuxtor\AppData\Local\Temp\ipykernel_8\4165299538.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exam

encoding

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CGPA                       10000 non-null  float64
 1   Internships                10000 non-null  int64  
 2   Projects                   10000 non-null  int64  
 3   Workshops/Certifications   10000 non-null  int64  
 4   AptitudeTestScore          10000 non-null  int64  
 5   SoftSkillsRating           10000 non-null  float64
 6   ExtracurricularActivities  10000 non-null  object 
 7   PlacementTraining          10000 non-null  object 
 8   SSC_Marks                  10000 non-null  int64  
 9   HSC_Marks                  10000 non-null  int64  
 10  PlacementStatus            10000 non-null  object 
 11  faculty                    10000 non-null  object 
dtypes: float64(2), int64(6), object(4)
memory usage: 937.6+ KB


In [10]:
df['CGPA'].value_counts() # bitta ustundagi nunique elementlarni chiqarib beradi 

CGPA
6.5    833
7.6    814
7.5    798
8.3    662
8.2    655
7.7    622
8.4    608
7.4    590
8.1    534
7.8    453
8.0    445
8.5    428
7.9    384
8.6    309
6.6    278
7.3    267
6.7    265
6.8    212
8.7    187
6.9    165
7.2    127
7.0    121
7.1     92
8.8     86
8.9     49
9.0     13
9.1      3
Name: count, dtype: int64

In [11]:
cardinality=df.nunique() # umumiy har bir ustundagi nunique elementlarni chiqarib beradi

In [12]:
cardinality

CGPA                         27
Internships                   3
Projects                      4
Workshops/Certifications      4
AptitudeTestScore            31
SoftSkillsRating             19
ExtracurricularActivities     2
PlacementTraining             2
SSC_Marks                    36
HSC_Marks                    32
PlacementStatus               2
faculty                       6
dtype: int64

In [13]:
df_encoded=df.copy()

In [14]:
for col in df_encoded.select_dtypes(include=['object']).columns:
    cardinality=df_encoded[col].nunique()
    if cardinality<=5:
        le=LabelEncoder()
        df_encoded[col]=le.fit_transform(df_encoded[col])
    else:
        df_encoded=pd.get_dummies(df_encoded,columns=[col],dtype=int,drop_first=True)
df=df_encoded

In [15]:
df.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty_Business Administration,faculty_Civil Engineering,faculty_Computer Science,faculty_Electrical Engineering,faculty_Mechanical Engineering
0,7.5,1,1,1,65,4.4,0,0,61,79,0,1,0,0,0,0
1,8.9,0,3,2,90,4.0,1,1,78,82,1,0,0,1,0,0
2,7.3,1,2,2,82,4.8,1,0,79,80,0,0,0,1,0,0
3,7.5,1,1,2,85,4.4,1,1,81,80,1,0,1,0,0,0
4,8.3,1,2,2,86,4.5,1,1,74,88,1,0,0,0,1,0


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CGPA                             10000 non-null  float64
 1   Internships                      10000 non-null  int64  
 2   Projects                         10000 non-null  int64  
 3   Workshops/Certifications         10000 non-null  int64  
 4   AptitudeTestScore                10000 non-null  int64  
 5   SoftSkillsRating                 10000 non-null  float64
 6   ExtracurricularActivities        10000 non-null  int64  
 7   PlacementTraining                10000 non-null  int64  
 8   SSC_Marks                        10000 non-null  int64  
 9   HSC_Marks                        10000 non-null  int64  
 10  PlacementStatus                  10000 non-null  int64  
 11  faculty_Business Administration  10000 non-null  int64  
 12  faculty_Civil Engin

In [16]:
num_col=df.select_dtypes(include=['int64','float64']).columns

In [17]:
num_col

Index(['CGPA', 'Internships', 'Projects', 'Workshops/Certifications',
       'AptitudeTestScore', 'SoftSkillsRating', 'ExtracurricularActivities',
       'PlacementTraining', 'SSC_Marks', 'HSC_Marks', 'PlacementStatus',
       'faculty_Business Administration', 'faculty_Civil Engineering',
       'faculty_Computer Science', 'faculty_Electrical Engineering',
       'faculty_Mechanical Engineering'],
      dtype='object')

In [18]:
scaler=StandardScaler()
df[num_col]=scaler.fit_transform(df[num_col])

In [19]:
df.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty_Business Administration,faculty_Civil Engineering,faculty_Computer Science,faculty_Electrical Engineering,faculty_Mechanical Engineering
0,-0.309343,-0.073889,-1.182822,-0.014598,-1.770910,0.184742,-1.188261,-1.651836,-0.782306,0.504368,-0.850439,2.370260,-0.335736,-0.819903,-0.333148,-0.417445
1,1.877818,-1.575689,1.121526,1.091319,1.292970,-0.787072,0.841566,0.605387,0.847618,0.840726,1.175863,-0.421895,-0.335736,1.219657,-0.333148,-0.417445
2,-0.621794,-0.073889,-0.030648,1.091319,0.312528,1.156555,0.841566,-1.651836,0.943496,0.616487,-0.850439,-0.421895,-0.335736,1.219657,-0.333148,-0.417445
3,-0.309343,-0.073889,-1.182822,1.091319,0.680194,0.184742,0.841566,0.605387,1.135251,0.616487,1.175863,-0.421895,2.978535,-0.819903,-0.333148,-0.417445
4,0.940464,-0.073889,-0.030648,1.091319,0.802749,0.427695,0.841566,0.605387,0.464106,1.513441,1.175863,-0.421895,-0.335736,-0.819903,3.001668,-0.417445


input va output qiymatlrni tanlab olish

In [20]:
x=df.drop(columns=['PlacementStatus'])

In [21]:
y=df['PlacementStatus']

In [22]:
df.head()

,CGPA,Internships,Projects,Workshops/Certifications,AptitudeTestScore,SoftSkillsRating,ExtracurricularActivities,PlacementTraining,SSC_Marks,HSC_Marks,PlacementStatus,faculty_Business Administration,faculty_Civil Engineering,faculty_Computer Science,faculty_Electrical Engineering,faculty_Mechanical Engineering
0,-0.309343,-0.073889,-1.182822,-0.014598,-1.770910,0.184742,-1.188261,-1.651836,-0.782306,0.504368,-0.850439,2.370260,-0.335736,-0.819903,-0.333148,-0.417445
1,1.877818,-1.575689,1.121526,1.091319,1.292970,-0.787072,0.841566,0.605387,0.847618,0.840726,1.175863,-0.421895,-0.335736,1.219657,-0.333148,-0.417445
2,-0.621794,-0.073889,-0.030648,1.091319,0.312528,1.156555,0.841566,-1.651836,0.943496,0.616487,-0.850439,-0.421895,-0.335736,1.219657,-0.333148,-0.417445
3,-0.309343,-0.073889,-1.182822,1.091319,0.680194,0.184742,0.841566,0.605387,1.135251,0.616487,1.175863,-0.421895,2.978535,-0.819903,-0.333148,-0.417445
4,0.940464,-0.073889,-0.030648,1.091319,0.802749,0.427695,0.841566,0.605387,0.464106,1.513441,1.175863,-0.421895,-0.335736,-0.819903,3.001668,-0.417445


datani qismlarga train,test,splitga ajratish

In [23]:
x_train,x_temp,y_train,y_temp=train_test_split(x,y,test_size=0.2,random_state=42)
x_test,x_val,y_test,y_val=train_test_split(x_temp,y_temp,test_size=0.5,random_state=42)

modelni tanlash

In [24]:
model=LinearRegression() #modelni tanlash       
Linear_model=model.fit(x_train,y_train) #modelni x va y train orqali o'rgatish
y_pred=Linear_model.predict(x_test) # train qilingan modelni x_test qilib oxirgi natija olinadi
mse=mean_squared_error(y_test,y_pred) # mse y_test(haqiqiy qiymatlar) va y_pred (predict qilingan qiymatlar) orasidagi farqni ta'minlaydi
r2=r2_score(y_test,y_pred) # modelning haqiqiy qiymatlarga qanchalik mos kelishini aniqlaydi, 0 va 1 oraliqda bo'lib, 1ga qanchalik yaqin bo'lsa natija shunchalik yaxshi chiqadi


In [25]:
mse

0.6151383891329372

In [26]:
r2

0.38693348506332437

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 16 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   CGPA                             10000 non-null  float64
 1   Internships                      10000 non-null  float64
 2   Projects                         10000 non-null  float64
 3   Workshops/Certifications         10000 non-null  float64
 4   AptitudeTestScore                10000 non-null  float64
 5   SoftSkillsRating                 10000 non-null  float64
 6   ExtracurricularActivities        10000 non-null  float64
 7   PlacementTraining                10000 non-null  float64
 8   SSC_Marks                        10000 non-null  float64
 9   HSC_Marks                        10000 non-null  float64
 10  PlacementStatus                  10000 non-null  float64
 11  faculty_Business Administration  10000 non-null  float64
 12  faculty_Civil Engin

In [ ]:
kf=KFold(n_splits=10,shuffle=True,random_state=42) # datani 10ta bo'lakka bo'lininb, shuffle aralashtirib olish, train va testga ajratilganda bir xiil natija olish

cv_scores=cross_val_score(Linear_model,x,y,cv=kf,scoring='neg_mean_squared_error')# 1) linear model oldindan aniqlangan model, 2) x,y model o'qitilayotgan ma'lumotlar 3)cv=kf 10 qismli cross validation # 4) scoring=neg_mean_squared_error modelni baholash mezoni : MSE, 5) cross_val_score MSE natijalrini manfiy qiymatda qaytaradi(sklearn barcha skorlash mezonlari oshishi kerak deb hisoblaydi)

cv_scores=np.sqrt(-cv_scores) # 1) cv_scores manfiy  MSE qiymatlardan iborat shuning uchun -cv_scores orqali musbat qiymat olinadi, 2) np.sqrt() orqali root mean squared error (RMSE) hisoblanadi

print("Cross-Validation Scores: ",cv_scores) # RMSE natijasi hisoblanadi
print("MeanCV score:",np.mean(cv_scores)) # o'rtacha RMSE hioblanadi

Cross=Validation Scores:  [0.76524452 0.77029382 0.78207065 0.77530673 0.76122458 0.78595863
 0.76085981 0.73277691 0.75868395 0.7681176 ]
MeanCV score: 0.7660537199151363


In [ ]:
print(np.mean(cv_scores)-mse) # o'rtacha RMSE va MSE o'rtasidagi farq

0.1509153307821991
